In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 16_000,
    "load_in_4bit": True,
    "enable_thinking": False,  # main_gpu interpretation
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║ ▓▓▓▓▓   ▓▓▓▓                                  ▓▓▓                ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓   ▓▓▓▓ ▓ ▓       ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓  ▓▁▁▓ ▓▓        ║
║  ▓▓   ▓▓ ▓▓  ▓    ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓   ▓    ▓         ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓   ▓▓▓▓ ▓         ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready ║
║ Faster builds, clearer flows, production-first                   ║
║                                                                  ║
║ Version: 0.1.0 | Python: 3.12.12                                 ║
║ OS: Linux 6.17.0-6-generic (x86_64)                              ║
║ Torch: 2.8.0   CUDA: yes (12.8)                                  ║
║ MPS: no (built: False)          

/home/nomi/micromamba/envs/LLMs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nomi/micromamba/envs/LLMs/lib/python3.12/importlib/__init__.py:90: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers does not work in RTX 50X, Blackwell GPUs as of yet. Please build from source via
```
pip install ninja
pip install -v --no-build-isolation -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers
```

🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-11-12 20:26:42 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-12 20:26:42 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO     | 2025-11-12 20:26:43 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.38s/it]

INFO     | 2025-11-12 20:26:47 | transformers.py:init_model | Transformers model initialized successfully.


In [3]:
# streaming response example
from IPython.display import display, Markdown, clear_output

system_prompt = "You are a joker."
user_prompt = """Tell me a light-hearted joke."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Ah, I've got one that's sure to make you smile!  

Why don't skeletons fight each other?  

Because they don't have the *guts*!  

*Knock-knock*  
*Who's there?*  
*My dad.*  
*My dad who?*  
*My dad... *jokes*...*  

(Okay, that one’s a bit of a *bone* to pick, but I’ll take the pun!) 😄

In [ ]:
# Structured Response examples
from pydantic import BaseModel

class Operation(BaseModel):
    num1: float
    num2: float
    operator: str
    sequence_number: int

class MathProblem(BaseModel):
    operations: list[Operation]

user_query = "What is 2+2?"

response = LLM.ask(user_query, output_schema=MathProblem)

print(response.parsed_output)


operations=[Operation(num1=2.0, num2=2.0, operator='+', sequence_number=1)]


## RAG wiring so the agent “understands” the Neurosurf codebase

You’ll ingest the repo once, then run a retriever to answer code questions. The Planner can call the retriever first to form a precise implementation plan.

### FileReader and Chunker Test

In [89]:
# scripts/index_repo_for_rag.py
from pathlib import Path
from neurosurfer.rag.ingestor import RAGIngestor
from neurosurfer.rag.chunker import Chunker
from neurosurfer.rag.filereader import FileReader
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
vs = ChromaVectorStore(collection_name="neurosurf-repo")
ing = RAGIngestor(
    embedder=embedder,
    vector_store=vs, 
    chunker=Chunker(), 
    file_reader=FileReader(),
    default_metadata={"collection": "neurosurf"}
)

root_dir = Path(os.getcwd()).parent.joinpath("neurosurfer")
ing.add_directory(root_dir)  # the repo root
print(ing.build())


INFO     | 2025-11-11 13:41:51 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-11 13:41:51 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-11-11 13:41:51 | posthog.py:__init__ | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Batches: 100%|██████████| 2/2 [00:00<00:00, 17.34it/s]


{'status': 'ok', 'sources': 101, 'chunks': 630, 'unique_chunks': 630, 'added': 630, 'finished_at': 1762854114.2439182}


## Graph AGENT

Prepare Tools

In [ ]:
from neurosurfer.tools.toolkit import Toolkit
from neurosurfer.tools.tool_spec import ToolSpec, ToolParam, ToolReturn
from neurosurfer.tools.base_tool import BaseTool, ToolResponse

# Simple Calculator Tool
class CalculatorTool(BaseTool):
    spec = ToolSpec(
        name="calculator",
        description="Perform basic arithmetic operations such as addition, subtraction, multiplication, and division.",
        when_to_use="Use this tool when you need to perform basic arithmetic operations.",
        inputs=[
            ToolParam(name="num1", type="float", description="The first number.", required=True),
            ToolParam(name="num2", type="float", description="The second number.", required=True),
            ToolParam(name="operation", type="string", description="The operation to perform strictly one of ['add', 'subtract', 'multiply', 'divide'].", required=True)
        ],
        returns=ToolReturn(type="float", description="The result of the arithmetic operation.")
    )

    def __init__(self, final_answer: bool = False):
        self.final_answer = final_answer

    def __call__(self, num1: float, num2: float, operation: str) -> ToolResponse:
        if operation not in ["add", "subtract", "multiply", "divide"]:
            return ToolResponse(
                final_answer=False,
                observation="Invalid operation. Supported operations are 'add', 'subtract', 'multiply', and 'divide'.",
                extras={}
            )
        
        if operation == "divide" and num2 == 0:
            return ToolResponse(
                final_answer=False,
                observation="Division by zero is not allowed.",
                extras={}
            )
        try:
            num1 = float(num1)
            num2 = float(num2)
            if operation == "add":
                result = num1 + num2
            elif operation == "subtract":
                result = num1 - num2
            elif operation == "multiply":
                result = num1 * num2
            elif operation == "divide":
                result = num1 / num2
        except Exception as e:
            return ToolResponse(
                final_answer=False,
                observation=f"An error occurred: {str(e)}",
                extras={}
            )
        
        return ToolResponse(
            final_answer=self.final_answer,
            observation=float(result),
            extras={}
        )

calculator_tool = CalculatorTool()
toolkit = Toolkit(tools=[calculator_tool])

print(calculator_tool.get_tool_description())


TypeError: Invalid tool type: CalculatorTool. Expected a subclass of BaseTool.

### YAML Flow

In [122]:
import asyncio
from graph import FlowRegistry, FlowSelector, FlowLoader, GraphExecutor
from graph.tracing import RichTracer

def run_async(coro):
    """
    In scripts: runs the coroutine immediately.
    In notebooks: returns the coroutine so you can `await` it.
    """
    try:
        loop = asyncio.get_running_loop()  # Jupyter: loop is already running
    except RuntimeError:
        return asyncio.run(coro)
    else:
        return coro  # caller must: result = await run_async(coro)

tracer = RichTracer()  # prints each span start/end

reg = FlowRegistry()
reg.register(FlowLoader.from_yaml("calc_and_explain_flow.yml"), version="v1")

sel = FlowSelector(registry=reg)
sel.add_rule(lambda q: any(k in q.lower() for k in ["%", "add", "sum", "multiply", "compute"]), "calc_and_explain@v1")

graph = sel.pick("Compute 3.5% of 12000 and explain", default_flow="calc_and_explain@v1")
# print(graph.nodes[0].inputs)
# print(graph.nodes[0].outputs)

executor = GraphExecutor(
    llm=LLM, 
    toolkit=toolkit, 
    max_concurrency=2,
    tracer=tracer, 
    trace_payloads=True,          # 👈 enable IO tracing
    trace_max_chars=400,          # preview size
    trace_redactor=lambda s: s.replace("sk-", "sk-***"),  # example redactor
)

result = await run_async(executor.run(
    graph,
    inputs={"prompt": "Compute 3.5% of 12000 and explain", "explanation_style": "brief"},
))
print("OK:", result.ok)
print("Result:", result.outputs["result"])
print("Explanation:", result.outputs["explanation"])

 ▶ node:rewrite {'kind': 'task', 'fn': 'llm.call'}

 ▶ node:rewrite:inputs {'phase': 'inputs', 'node': 'rewrite', 'io_preview': {'query': "You are a math problem 
rewriter.\nPrompt: Compute 3.5% of 12000 and explain\nStyle: brief\n\nRewrite the problem in a different way, but 
keep the core math operation the same.\nAvoid extra fluff. Operation should be one of: ['add', 'subtract', 
'multiply', 'divide']\n"}}

 ◀ node:rewrite:inputs took 0.001s; error=False

llm.call Node(id='rewrite', kind='task', fn='llm.call', inputs={'query': "You are a math problem rewriter.\nPrompt: ${inputs.prompt}\nStyle: ${inputs.explanation_style}\n\nRewrite the problem in a different way, but keep the core math operation the same.\nAvoid extra fluff. Operation should be one of: ['add', 'subtract', 'multiply', 'divide']\n"}, outputs={'num1': 'float', 'num2': 'float', 'operation': 'str'}, map_over=None, policy=NodePolicy(retries=1, timeout_s=30, backoff='exponential', backoff_base=0.6, budget={'max_new_tokens': 180, 'temperature': 0.2}, model_hint=None, concurrency_group=None))
payload {'query': "You are a math problem rewriter.\nPrompt: Compute 3.5% of 12000 and explain\nStyle: brief\n\nRewrite the problem in a different way, but keep the core math operation the same.\nAvoid extra fluff. Operation should be one of: ['add', 'subtract', 'multiply', 'divide']\n"}
outputs {'num1': 'float', 'num2': 'float', 'operation': 'str'} <class 'dict'>
sys-----------:
 You are a

 ▶ node:rewrite:outputs {'phase': 'outputs', 'node': 'rewrite', 'io_preview': {'num1': '12000.0', 'num2': '3.5', 
'operation': 'multiply'}}

...

 ◀ node:compute:inputs took 0.001s; error=False

 ▶ node:compute:outputs {'phase': 'outputs', 'node': 'compute', 'io_preview': {'calculation': 'None'}}

 ◀ node:compute:outputs took 0.000s; error=False

 ◀ node:compute took 0.002s; error=False

OK: True
Result: None
Explanation: 12000 multiplied by 3.5 equals 42000. This result makes sense because multiplying 12000 by 3 gives 36000, and multiplying 12000 by 0.5 gives 6000, so adding those together gives 42000.


### Python API version (no YAML)

In [9]:
import asyncio
from graph import Graph, Node, NodePolicy, GraphConfig, GraphExecutor
from neurosurfer.tools import Toolkit
from neurosurfer.models.chat_models.openai import OpenAIModel

# Reuse your existing toolkit + model
llm = LLM  # already created in your environment
tk = toolkit

graph = Graph(
    name="calc_and_explain",
    config=GraphConfig(max_concurrency=2),
    inputs_schema={"prompt": str},
    nodes=[
        Node(
            id="rewrite",
            fn="general_query_assistant",  # adjust name if needed
            inputs={
                # swap "query" -> "prompt" if your tool expects "prompt"
                "query": (
                    "You will receive a user request. Extract a SINGLE pure arithmetic expression that can be "
                    "evaluated by a calculator (e.g., '(42 * 7) - 5^2' or '0.035 * 12000').\n"
                    "- Do NOT include explanations.\n"
                    "- Return ONLY the expression as plain text.\n\n"
                    "User request:\n${inputs.prompt}"
                )
            },
            outputs=["num1", "num2", "operation"],
            policy=NodePolicy(
                retries=1,
                timeout_s=30,
                budget={"max_new_tokens": 128, "temperature": 0.1},
            ),
        ),
        Node(
            id="compute",
            fn="calculator",
            inputs={"num1": "${rewrite.num1}", "num2": "${rewrite.num2}", "operation": "${rewrite.operation}"},
            outputs=["text"],
            policy=NodePolicy(retries=0, timeout_s=15),
        ),
        Node(
            id="explain",
            fn="general_query_assistant",
            inputs={
                "query": (
                    "Original request: ${inputs.prompt}\n"
                    "Calculator result: ${compute.text}\n\n"
                    "Write a brief, user-friendly explanation of the result (one short paragraph)."
                )
            },
            outputs=["text"],
            policy=NodePolicy(
                retries=1,
                timeout_s=30,
                budget={"max_new_tokens": 180, "temperature": 0.2},
            ),
        ),
    ],
    outputs={"answer": "${explain.text}"},
)

executor = GraphExecutor(llm=llm, toolkit=tk, max_concurrency=2)

result = await run_async(
    executor.run(graph, inputs={"prompt": "Compute 3.5% of 12000 and explain"}, stream=True)
)

print("OK:", result.ok)
print("Answer:\n", result.outputs["answer"])


OK: True
Answer:
 The calculator result for your request is ${compute.text}. This means that after performing the calculation based on your input, the final answer is ${compute.text}. Let me know if you need further assistance!


### Planner-based path (using the YAML as a skeleton)

In [ ]:
import asyncio, tempfile, pathlib
from graph import PlannerAgent, FlowLoader, GraphExecutor

# 1) Write the YAML to a temp file (only for this demo)
yaml_text = r"""
name: calc_and_explain
inputs:
  prompt: str
config:
  max_concurrency: 2
nodes:
  - id: rewrite
    kind: task
    fn: general_query_assistant
    inputs:
      query: |
        You will receive a user request. Extract a SINGLE pure arithmetic expression that can be
        evaluated by a calculator (e.g., "(42 * 7) - 5^2" or "0.035 * 12000").
        - Do NOT include explanations.
        - Return ONLY the expression as plain text.

        User request:
        ${inputs.prompt}
    outputs: ["text"]
    policy: { retries: 1, timeout_s: 30, budget: { max_new_tokens: 128, temperature: 0.1 } }

  - id: compute
    kind: task
    fn: calculator
    inputs: { expression: ${rewrite.text} }
    outputs: ["text"]

  - id: explain
    kind: task
    fn: general_query_assistant
    inputs:
      query: |
        Original request: ${inputs.prompt}
        Calculator result: ${compute.text}

        Write a brief, user-friendly explanation of the result (one short paragraph).
    outputs: ["text"]
    policy: { retries: 1, timeout_s: 30, budget: { max_new_tokens: 180, temperature: 0.2 } }

outputs: { answer: ${explain.text} }
""".strip()

tmp = pathlib.Path(tempfile.gettempdir()) / "calc_and_explain.yml"
tmp.write_text(yaml_text)

# 2) Use the planner with a skeleton (so it returns your YAML-based Graph)
planner = PlannerAgent(llm=LLM)  # LLM not used when skeleton is set
graph = planner.plan_from_query(query="Compute 3.5% of 12000 and explain", skeleton=str(tmp))

# 3) Execute
executor = GraphExecutor(llm=LLM, toolkit=toolkit, max_concurrency=2)
result = asyncio.run(executor.run(graph, inputs={"prompt": "Compute 3.5% of 12000 and explain"}))

print("OK:", result.ok)
print(result.outputs["answer"])


Test ToolsRouterAgent

In [8]:
query = "Perform the calculation 20 * 90"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Using tool: calculator
INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Raw inputs: {'num1': 20.0, 'num2': 90.0, 'operation': 'multiply'}
1800.0

In [9]:
query = "Tell me a light-hearted joke!"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Using tool: general_query_assistant
INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Raw inputs: {'query': 'Tell me a light-hearted joke!'}
Why don't skeletons fight each other? They don't have the guts!None

## ReactAgent

In [21]:
from neurosurfer.agents.react import ReActAgent, ReActConfig

react_agent = ReActAgent(
    toolkit=toolkit,
    llm=LLM,
    specific_instructions="Always be concise in your answers. Break the task into steps if needed.",
    config=ReActConfig(
        temperature=0.7,
        max_new_tokens=4096,
        allow_input_pruning=True,
        repair_with_llm=True,
        skip_special_tokens=True,
        verbose=True
    )
)

# print(react_agent._system_prompt())
TASK = """Calculate 300 - 300. Then tell me a light-hearted joke about that result."""

for chunk in react_agent.run(TASK):
    print(chunk, end="")




[🧠] Chain of Thoughts...
Thought: I will first calculate 300 - 300 using the calculator tool, and then I will use the general_query_assistant tool to find a light-hearted joke about the result.

Action: {
  "tool": "calculator",
  "inputs": {
    "num1": 300,
    "num2": 300,
    "operation": "subtract"
  },
  "final_answer": false
}

[🔧] Tool: calculator
[📤] Inputs: {'num1': 300, 'num2': 300, 'operation': 'subtract'}

Observation:



[🧠] Chain of Thoughts...
Thought: The result of the calculation is 0. Now, I will use the general_query_assistant tool to find a light-hearted joke about the result.

Action: {
  "tool": "general_query_assistant",
  "inputs": {
    "query": "Tell me a light-hearted joke about the number 0."
  },
  "final_answer": true
}

[🔧] Tool: general_query_assistant
[📤] Inputs: {'query': 'Tell me a light-hearted joke about the number 0.'}

Why did the number 0 break up with the number 8?  
Because it found someone more "8" (8) than a zero!

Observation: Why did the number 0 break up with the number 8?  
Because it found someone more "8" (8) than a zero!



[🧠] Chain of Thoughts...
Thought: The calculation result is 0, and the joke provided is ready. The final answer is complete.

Final Answer: The result of 300 - 300 is 0. Here's a light-hearted joke about it: Why did the number 0 break up with the number 8? Because it found someone more "8" (8) than a zero!